# Detail Explanation for below code
The `SingleServerQueue` class represents the single-server queuing system. It is initialized with the simulation environment (`env`), the average arrival rate of customers per unit of time (`arrival_rate`), and the average service rate of the server per unit of time (`service_rate`). The class also has attributes to keep track of the queue length (`queue_length`) and the waiting times of customers (`waiting_times`).

The `customer_arrival` method is a generator function that simulates customer arrivals. It runs indefinitely in a loop (`while True`). Inside the loop, the function `yield` pauses the execution for a random time interval based on an exponential distribution with the specified arrival rate (`random.expovariate(self.arrival_rate)`). This simulates the inter-arrival time between consecutive customers. After a customer arrives, we increment the queue length by one (`self.queue_length += 1`), and then we start the customer service process using `self.env.process(self.customer_service(self.env.now))`. The `self.env.now` captures the current simulation time, representing the time of customer arrival.

The `customer_service` method simulates the service process for a customer. It starts by capturing the arrival time passed as `arrival_time`. The service time is sampled from an exponential distribution with the specified service rate (`random.expovariate(self.service_rate)`). We then yield `self.env.timeout(service_time)`, which represents the duration of the service. After the service is completed, the departure time is captured as the current simulation time (`self.env.now`), and we calculate the waiting time as `waiting_time = departure_time - arrival_time`. We decrement the queue length by one (`self.queue_length -= 1`) and add the waiting time to the list of waiting times for customers (`self.waiting_times.append(waiting_time)`).

In the main simulation part, we define the arrival rate (`arrival_rate`), service rate (`service_rate`), and the simulation time (`simulation_time`). We then create a simulation environment (`env`) using `simpy.Environment()`.
Next, we create an instance of the `SingleServerQueue` class called `queue_system`, passing the simulation environment and the arrival and service rates to it.
We then start the customer arrival process using `env.process(queue_system.customer_arrival())`, which initiates the queuing system simulation.
The simulation runs until the specified `simulation_time` using `env.run(until=simulation_time)`.
After the simulation is complete, we calculate the average waiting time of customers using the recorded waiting times and print the result.
Finally, we plot the waiting time distribution using `matplotlib`. The histogram shows the probability density of waiting times for customers in the single-server queuing system.

In [ ]:
import simpy
import random
import matplotlib.pyplot as plt

class SingleServerQueue:
    def __init__(self, env, arrival_rate, service_rate):
        self.env = env
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.queue_length = 0
        self.waiting_times = []

    def customer_arrival(self):
        while True:
            yield self.env.timeout(random.expovariate(self.arrival_rate))
            self.queue_length += 1
            self.env.process(self.customer_service(self.env.now))  # Capture arrival time

    def customer_service(self, arrival_time):
        service_time = random.expovariate(self.service_rate)
        yield self.env.timeout(service_time)
        departure_time = self.env.now
        self.queue_length -= 1
        waiting_time = departure_time - arrival_time
        self.waiting_times.append(waiting_time)

if __name__ == '__main__':
    arrival_rate = 0.5  # Average arrival rate of customers per unit of time
    service_rate = 0.6  # Average service rate of the server per unit of time
    simulation_time = 1000

    env = simpy.Environment()
    queue_system = SingleServerQueue(env, arrival_rate, service_rate)
    env.process(queue_system.customer_arrival())
    env.run(until=simulation_time)

    # Calculate and print average waiting time
    average_waiting_time = sum(queue_system.waiting_times) / len(queue_system.waiting_times)
    print("Average Waiting Time:", average_waiting_time)

    # Plotting the waiting time distribution
    plt.hist(queue_system.waiting_times, bins=30, density=True, alpha=0.7, color='blue')
    plt.xlabel('Waiting Time')
    plt.ylabel('Probability Density')
    plt.title('Waiting Time Distribution in Single-Server Queuing System')
    plt.show()


## Alternative

# Detail Explanation for below code
We set the random seed for reproducibility (`RANDOM_SEED`), the simulation time (`SIM_TIME`), and the service time for the server (`SERVICE_TIME`).

The `SingleServerQueue` class represents the single-server queuing system. It is initialized with the simulation environment (`env`) and the service time of the server (`service_time`). The class uses `simpy.Resource` to create a resource representing the server with a capacity of 1 (meaning only one customer can be served at a time). The `service` method is a generator function that simulates the service process for a customer. It yields a `timeout` event based on an exponential distribution with the inverse of the service time (`1/self.service_time`), which represents the duration of the service.

The `customer_arrivals` function simulates the arrival of customers to the queuing system. It runs indefinitely in a loop (`while True`). Inside the loop, it increments the customer index `i` by one to keep track of the number of arriving customers. The function then yields a `timeout` event based on an exponential distribution with a rate of `1/5`, which represents the average time between customer arrivals. After the timeout, it starts the `customer` process, passing the simulation environment (`env`), the customer name (`f'Customer {i}'`), and the `queue` object representing the single-server queue.

The `customer` function simulates the behavior of a customer in the queuing system. When a customer arrives, it prints a message indicating the arrival time and the customer's name. It then requests to use the server using `queue.server.request()` as a context manager. If the server is available, the customer is served. It prints a message when the customer starts being served, and it yields the service process by calling `env.process(queue.service())`. The `queue.service()` generator function is called, which represents the service time of the customer. After the service is completed, the customer prints a message indicating the finish time.

In the main simulation part, we set the random seed using `random.seed(RANDOM_SEED)` for reproducibility. We create the simulation environment `env` using `simpy.Environment()`. We also create an instance of the `SingleServerQueue` class called `queue`, passing the simulation environment and the service time `SERVICE_TIME`.

Next, we start the customer arrival process using `env.process(customer_arrivals(env, queue))`. This initiates the arrival of customers to the queuing system.

The simulation runs until the specified `SIM_TIME` using `env.run(until=SIM_TIME)`. During the simulation, the arrivals and services of customers are simulated, and their interactions with the single-server queuing system are recorded and printed as messages.

The output will show the arrival, service start, and service finish times for each customer, which helps analyze the performance of the queuing system

In [ ]:
import random
import simpy

RANDOM_SEED = 42
SIM_TIME = 1000
SERVICE_TIME = 5

class SingleServerQueue:
    def __init__(self, env, service_time): 
        self.server = simpy.Resource(env, capacity=1)
        self.service_time = service_time
    def service(self):
        yield env.timeout (random.expovariate(1/self.service_time))

def customer_arrivals(env, queue):
    i = 0
    while True:
        i += 1 
        yield env.timeout(random.expovariate(1/5)) 
        env.process(customer (env, f'Customer {i}', queue))

def customer(env, name, queue): 
    print (f" {env.now}: {name} arrived")
    with queue.server.request() as request:
        yield request
        print(f" {env.now}: {name} being served")
        yield env.process(queue.service())
        print(f"{env.now}: {name} finished being served")

# Setup and start the simulation 
random.seed(RANDOM_SEED)
env = simpy.Environment()
queue = SingleServerQueue(env, SERVICE_TIME) 
env.process(customer_arrivals (env, queue))
env.run(until=SIM_TIME)

## Another two Alternative
https://towardsdatascience.com/simulating-a-queuing-system-in-python-8a7d1151d485

https://medium.com/analytics-vidhya/simulating-a-single-server-queuing-system-in-python-f8e32578749f